# My Awesome Notebook

Cell outputs are preserved.

In [1]:
pre_executed_cell = 1
pre_executed_cell + 2

3

## Installing pure python packages from PyPI

Packages that are pure python wheels (eg no C extensions) can be installed liked normal from PyPI:

In [ ]:
%pip install python-dateutil

In [ ]:
from dateutil.parser import parse

parse("Sat Oct 11 17:13:46 UTC 2003")

## Installing WASM wheels from pyodide

Some packages require native libraries written in C or rust.
For example, numpy and pandas.
Some of these packages are built as WASM wheels and available in jupyterlite,
with no build-time step required from you.

For example, the vega_datasets package depends on pandas:

In [ ]:
%pip install altair vega_datasets

Then you can import them like normal:

In [ ]:
import altair as alt  # ty: ignore[unresolved-import]
from vega_datasets import data  # ty: ignore[unresolved-import]

source = data.cars()
brush = alt.selection_interval()
points = (
    alt.Chart(source, title="Drag an Area")
    .mark_point()
    .encode(
        x="Horsepower:Q",
        y="Miles_per_Gallon:Q",
        color=alt.when(brush).then("Origin:N").otherwise(alt.value("lightgray")),
    )
    .add_params(brush)
)

bars = (
    alt.Chart(source)
    .mark_bar()
    .encode(y="Origin:N", color="Origin:N", x="count(Origin):Q")
    .transform_filter(brush)
)

points & bars

## Installing from a wheel bundled at docs build time

This is the `mkdocs.yml` for this notebook:

```yaml
# more stuff up here
plugins:
  - jupyterlite:
      notebook_patterns: ['*.ipynb']
      wheels:
        - url: "https://files.pythonhosted.org/packages/44/53/5ae386965f4fc73a06a868dc92a22e163e0b3a5c6fb449f67172746a912c/cowsay-3.0-py2.py3-none-any.whl"
        - command: "cd src/package_not_on_pypi/ && uv build --out-dir {wheels_dir}"
```

See the list of `wheels`? Each entry in that list can either be a `url` or a `command`.

The `url` type is straightforward.

This plugin runs the `command` in the shell, with "{wheels_dir}" replaced with a temp directory.
The command is expected to place 0-N files ending with `.whl` in that directory,
which will be included in the jupyterlite build.

Because of that, we can install with `%pip`, and the underlying micropip
will be able to find that wheel, instead of looking for it on PyPI.

In [ ]:
%pip install package-not-on-pypi

In [ ]:
import package_not_on_pypi

package_not_on_pypi.hello()  # ty: ignore[unresolved-attribute]